Оценка RAG

In [2]:
from typing import List
import os
import json
import numpy as np
from react_agent.tools import search  # ваш инструмент с Qdrant + fastembed
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy

class SimpleRAG:

    def __init__(self, model_name: str = "gpt-4o"):
        self.llm = ChatOpenAI(model=model_name)
        self.embeddings = OpenAIEmbeddings()

    def retrieve(self, query: str, top_k: int = 10) -> List[str]:
        """
        Вызов Qdrant-инструмента search() для получения до top_k фрагментов контекста.
        """
        raw = search(query)
        # search() возвращает строки, разделённые сериями ===== и заголовками источников
        # Разбиваем по заголовку каждого результата
        parts = [part.strip() for part in raw.split("==========Актуально в период:") if part.strip()]
        # Берём первые top_k результатов
        return parts[:top_k]


ResponseHandlingException: [Errno 111] Connection refused

Создание датасета

In [ ]:
dataset = []
    for q in questions:
        contexts = rag.retrieve(q)
        #response = rag.answer(q, contexts)
        dataset.append({
            "user_input": q,
            "retrieved_contexts": contexts,
            #"response": response
        })

In [ ]:
from ragas import EvaluationDataset
eval_ds = EvaluationDataset.from_list(dataset)


Оценка

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy

evaluator = LangchainLLMWrapper(rag.llm)
metrics = [LLMContextRecall(), Faithfulness(), ResponseRelevancy()]
results = evaluate(
    dataset=eval_ds,
    metrics=metrics,
    llm=evaluator
)
print(results)


In [4]:
import requests
import os

def yandex_generative_search(question: str, 
                            api_key: str = None, 
                            iam_token: str = None, 
                            folder_id: str = None) -> str:
    """
    Функция для генерации ответов через Yandex Search API
    
    :param question: Вопрос/запрос пользователя
    :param api_key: API-ключ сервисного аккаунта (альтернатива iam_token)
    :param iam_token: IAM-токен для аутентификации
    :param folder_id: Идентификатор каталога
    :return: Сгенерированный ответ
    """
    # URL API из документации
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    
    # Заголовки запроса
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {iam_token}" if iam_token else f"Api-Key {api_key}"
    }
    
    # Тело запроса
    payload = {
        "modelUri": f"gpt://{folder_id}/yandexgpt-lite",
        "completionOptions": {
            "stream": False,
            "temperature": 0.6,
            "maxTokens": "2000"
        },
        "site": {
            "site": [
            "https://mai.ru",
            ]
        },
        "messages": [
            {
                "role": "user",
                "text": question
            }
        ]
    }
    
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        
        # Парсинг ответа
        result = response.json()
        return result['result']['alternatives'][0]['message']['text']
        
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None
    except KeyError:
        print("Error parsing response")
        return None

In [8]:
response = yandex_generative_search(
    question="Как приготовить борщ?",
    folder_id="b1gst3c7cskk2big5fqn",
    iam_token="t1.9euelZqQyIqPnceOz8yKkpzHkcmXy-3rnpWaz5eZyJmOiprIx4yRzJiRlJHl8_c-PDY_-e9Abx09_N3z935qMz_570BvHT38zef1656Vmsaax5jMnZKXypiOms-Rypaa7_zN5_XrnpWaj56MmoyTyJrOkZiYypqMnInv_cXrnpWaxprHmMydkpfKmI6az5HKlpo.Lihm-tzAHBhTiM7jAXN6-FbjQGT7gGVPCVBFtNrNIM5ScdHBFF1NJ-R8CIVXInE4FDEQj91hprxrlOGkwhKZCg")

print(response)

Для приготовления борща вам понадобятся следующие ингредиенты:

* говядина на кости — 500 г;
* свёкла — 2 шт.;
* картофель — 4 шт.;
* морковь — 1 шт.;
* лук репчатый — 1 шт.;
* капуста белокочанная — 300 г;
* томатная паста — 2 ст. л.;
* чеснок — 2 зубчика;
* лавровый лист — 2 шт.;
* растительное масло — для жарки;
* соль, перец, зелень — по вкусу.

**Приготовление:**

1. Сварите бульон. Для этого положите мясо в кастрюлю, залейте холодной водой и поставьте на огонь. После закипания снимите пену и уменьшите огонь. Варите бульон около 1,5–2 часов, периодически снимая пену.
2. Пока варится бульон, подготовьте овощи. Свёклу нарежьте тонкой соломкой или натрите на крупной тёрке. Морковь и лук очистите и нарежьте: морковь — соломкой или натрите на крупной тёрке, лук — мелкими кубиками.
3. На сковороде разогрейте растительное масло и обжарьте лук до золотистого цвета. Затем добавьте морковь и жарьте ещё несколько минут.
4. Добавьте в сковороду свёклу, томатную пасту, соль, перец и немного бу